In [6]:
''' 
    Trivariate setting
    ODE solver
'''

# Choose timepoint and number of intervals to partition interval [0,some_t]
some_t = 50
some_ivls = 10**3


''' First moments - obtained from ODE solver package '''

def ODE_L_tri_sol_1st(t, ivls):
    
    def ODE_L_tri_1st(L,u):
        dLdu = np.dot(B_tri-D_alpha_tri, L) + L_01_3dim
        return dLdu
    
    t_vec_ODE = np.linspace(0,t,ivls)
    return integrate.odeint(ODE_L_tri_1st, l_inf_tri, t_vec_ODE)

# To enhance computational efficiency, interpolate first moments of L to use as input for ODE of Q
# And that of Q as input for the second order moments later 
# (This is to avoid nested systems of ODEs)

def get_ipltd_function_1st(t, ivls, func):
    sol_ODE = func(t,ivls)
    timespace = np.linspace(0,t,ivls)
    
    # Interpolate all first order moments
    M1 = interp1d(timespace, sol_ODE[:,0], kind='cubic',fill_value='extrapolate')
    M2 = interp1d(timespace, sol_ODE[:,1], kind='cubic',fill_value='extrapolate')
    M3 = interp1d(timespace, sol_ODE[:,2], kind='cubic',fill_value='extrapolate')
    return M1, M2, M3

L1_ipltd, L2_ipltd, L3_ipltd = get_ipltd_function_1st(some_t, some_ivls, ODE_L_tri_sol_1st)

def L_ipltd_1st(t):
    return np.array([L1_ipltd(t), L2_ipltd(t), L3_ipltd(t)])

def ODE_Q_tri_sol_1st(t, ivls):
    t_vec_ODE = np.linspace(0, t, ivls)
    def ODE_Q_tri_1st(Q,u):
        dQdu = -np.dot(D_mu_tri, Q) + L_ipltd_1st(u)
        return dQdu
    return integrate.odeint(ODE_Q_tri_1st, np.zeros(3), t_vec_ODE)

Q1_ipltd, Q2_ipltd, Q3_ipltd = get_ipltd_function_1st(some_t, some_ivls, ODE_Q_tri_sol_1st)

def Q_ipltd_1st(t):
    return np.array([Q1_ipltd(t), Q2_ipltd(t), Q3_ipltd(t)])



''' Second order moments - obtained from ODE solver package 
    Note: we need additional wrapper of scipy's odeint for matrix-valued ODEs, namely the _odeintw.py file
    Source: https://github.com/WarrenWeckesser/odeintw'''

def ODE_L_tri_sol_2nd(t, ivls):
    t_vec_ODE = np.linspace(0,t,ivls)
    BDa = B_tri - D_alpha_tri

    def ODE_L_tri_2nd(L,u):
        L_01 = np.diag(L_ipltd_1st(u))
        l_inf_outer1 = np.outer(l_inf_tri, L_ipltd_1st(u))
        l_inf_outer2 = np.outer(L_ipltd_1st(u), l_inf_tri)
        B_temp = np.ones((3,3))
        np.fill_diagonal(B_temp,2)
        dLdu = np.dot(BDa, L) + np.dot(L, BDa.T) + np.multiply(B_temp,np.dot(np.dot(B_tri,L_01),B_tri.T)) + np.dot(D_alpha_tri, l_inf_outer1)  + np.dot(l_inf_outer2, D_alpha_tri)
        return dLdu 
    
    return _odeintw.odeintw(ODE_L_tri_2nd, L_02_3dim, t_vec_ODE)

# To enhance computational efficiency, interpolate second moments of L to use as input for ODE of Q*L and Q^{[2]}
def get_ipltd_function_2nd(t, ivls, func):
    # Obtain the solution by solving the ODE
    sol_ODE = func(t, ivls)
    timespace = np.linspace(0,t,ivls)
    
    # Interpolate all moments
    M11 = interp1d(timespace, sol_ODE[:,0,0], kind='cubic',fill_value='extrapolate')
    M12 = interp1d(timespace, sol_ODE[:,0,1], kind='cubic',fill_value='extrapolate')
    M13 = interp1d(timespace, sol_ODE[:,0,2], kind='cubic',fill_value='extrapolate')
    M21 = interp1d(timespace, sol_ODE[:,1,0], kind='cubic',fill_value='extrapolate')
    M22 = interp1d(timespace, sol_ODE[:,1,1], kind='cubic',fill_value='extrapolate')
    M23 = interp1d(timespace, sol_ODE[:,1,2], kind='cubic',fill_value='extrapolate')
    M31 = interp1d(timespace, sol_ODE[:,2,0], kind='cubic',fill_value='extrapolate')
    M32 = interp1d(timespace, sol_ODE[:,2,1], kind='cubic',fill_value='extrapolate')
    M33 = interp1d(timespace, sol_ODE[:,2,2], kind='cubic',fill_value='extrapolate')
    
    return [M11, M12, M13], [M21,M22,M23], [M31,M32,M33]

L1_ipltd_2nd, L2_ipltd_2nd, L3_ipltd_2nd = get_ipltd_function_2nd(some_t, some_ivls, ODE_L_tri_sol_2nd)

# Define interpolated function to avoid doing repeated interpolation
def L_ipltd_2nd(t):
    L1 = np.array([L1_ipltd_2nd[0](t), L1_ipltd_2nd[1](t), L1_ipltd_2nd[2](t)])
    L2 = np.array([L2_ipltd_2nd[0](t), L2_ipltd_2nd[1](t), L2_ipltd_2nd[2](t)])
    L3 = np.array([L3_ipltd_2nd[0](t), L3_ipltd_2nd[1](t), L3_ipltd_2nd[2](t)])
    return np.array([L1, L2, L3])


''' Second order mixed-moments Q-lambda '''

def ODE_LQ_tri_sol_2nd(t,ivls):
    t_vec_ODE = np.linspace(0,t,ivls)
    init = np.zeros((3,3))
    
    def ODE_LQ_tri_2nd(QL,u):
        L_01 = L_ipltd_1st(u)
        Q_01 = Q_ipltd_1st(u)
        L_02 = L_ipltd_2nd(u)
        dQLdu = np.dot(B_tri - D_alpha_tri, QL) - np.dot(QL, D_mu_tri) + L_02 + np.outer(L_01_3dim, Q_01) + np.dot(B_tri, np.diag(L_01))
        return dQLdu
                       
    return _odeintw.odeintw(ODE_LQ_tri_2nd, init, t_vec_ODE)


L1Q_ipltd_2nd, L2Q_ipltd_2nd, L3Q_ipltd_2nd = get_ipltd_function_2nd(some_t, some_ivls, ODE_LQ_tri_sol_2nd)

def LQ_ipltd_2nd(t):
    L1Q = np.array([L1Q_ipltd_2nd[0](t), L1Q_ipltd_2nd[1](t), L1Q_ipltd_2nd[2](t)])
    L2Q = np.array([L2Q_ipltd_2nd[0](t), L2Q_ipltd_2nd[1](t), L2Q_ipltd_2nd[2](t)])
    L3Q = np.array([L3Q_ipltd_2nd[0](t), L3Q_ipltd_2nd[1](t), L3Q_ipltd_2nd[2](t)])
    return np.array([L1Q, L2Q, L3Q])


''' Second order reduced Q moments '''

def ODE_Q_tri_sol_2nd(t, ivls):
    t_vec_ODE = np.linspace(0,t,ivls)
    init = np.zeros((3,3))
    
    def ODE_Q_tri_2nd(Q,u):
        LQ = LQ_ipltd_2nd(u)
        dQdu = -np.dot(D_mu_tri, Q) - np.dot(Q, D_mu_tri) + LQ + LQ.T
        return dQdu
    
    return _odeintw.odeintw(ODE_Q_tri_2nd, init, t_vec_ODE)



In [7]:
''' 
    Bivariate setting
    ODE solver
'''

# Choose timepoint and number of intervals to partition interval [0,some_t]
some_t = 50
some_ivls = 10**3

''' First moments - obtained from ODE solver package '''

def ODE_L_bi_sol_1st(t, ivls):
    
    def ODE_L_bi_1st(L,u):
        dLdu = np.dot(B_bi-D_alpha_bi, L) + L_01_2dim
        return dLdu
    
    t_vec_ODE = np.linspace(0,t,ivls)
    return integrate.odeint(ODE_L_bi_1st, l_inf_bi, t_vec_ODE)

# To enhance computational efficiency, interpolate first moments of L to use as input for ODE of Q
# And that of Q as input for the second order moments later 
def get_ipltd_function_1st_bi(t, ivls, func):
    sol_ODE = func(t,ivls)
    timespace = np.linspace(0,t,ivls)
    
    # Interpolate all first order moments
    M1 = interp1d(timespace, sol_ODE[:,0], kind='cubic',fill_value='extrapolate')
    M2 = interp1d(timespace, sol_ODE[:,1], kind='cubic',fill_value='extrapolate')
    return M1, M2

L1_ipltd_bi, L2_ipltd_bi = get_ipltd_function_1st_bi(some_t, some_ivls, ODE_L_bi_sol_1st)

def L_ipltd_1st_bi(t):
    return np.array([L1_ipltd_bi(t), L2_ipltd_bi(t)])

def ODE_Q_bi_sol_1st(t, ivls):
    t_vec_ODE = np.linspace(0, t, ivls)
    def ODE_Q_bi_1st(Q,u):
        dQdu = -np.dot(D_mu_bi, Q) + L_ipltd_1st_bi(u)
        return dQdu
    return integrate.odeint(ODE_Q_bi_1st, np.zeros(2), t_vec_ODE)

Q1_ipltd_bi, Q2_ipltd_bi = get_ipltd_function_1st_bi(some_t, some_ivls, ODE_Q_bi_sol_1st)

def Q_ipltd_1st_bi(t):
    return np.array([Q1_ipltd_bi(t), Q2_ipltd_bi(t)])




''' Second order moments lambda - obtained from ODE solver package 
    Note: we need additional wrapper of scipy's odeint for matrix-valued ODEs, namely the _odeintw.py file
    Source: https://github.com/WarrenWeckesser/odeintw'''

def ODE_L_bi_sol_2nd(t, ivls):
    t_vec_ODE = np.linspace(0,t,ivls)
    BDa_bi = B_bi - D_alpha_bi

    def ODE_L_bi_2nd(L,u):
        L_01 = np.diag(L_ipltd_1st_bi(u))
        l_inf_outer1 = np.outer(l_inf_bi, L_ipltd_1st_bi(u))
        l_inf_outer2 = np.outer(L_ipltd_1st_bi(u), l_inf_bi)
        B_temp = np.ones((2,2))
        np.fill_diagonal(B_temp,2)
        
        dLdu = np.dot(BDa_bi, L) + np.dot(L, BDa_bi.T) + np.multiply(B_temp,np.dot(np.dot(B_bi,L_01),B_bi.T)) + np.dot(D_alpha_bi, l_inf_outer1)  + np.dot(l_inf_outer2, D_alpha_bi)
        return dLdu
    
    return _odeintw.odeintw(ODE_L_bi_2nd, L_02_2dim, t_vec_ODE)



# To enhance computational efficiency, interpolate second moments of L to use as input for ODE of Q*L and Q^{[2]}
def get_ipltd_function_2nd_bi(t, ivls, func):
    # Obtain the solution by solving the ODE
    sol_ODE = func(t, ivls)
    timespace = np.linspace(0,t,ivls)
    
    # Interpolate all moments
    M11 = interp1d(timespace, sol_ODE[:,0,0], kind='cubic',fill_value='extrapolate')
    M12 = interp1d(timespace, sol_ODE[:,0,1], kind='cubic',fill_value='extrapolate')
    M21 = interp1d(timespace, sol_ODE[:,1,0], kind='cubic',fill_value='extrapolate')
    M22 = interp1d(timespace, sol_ODE[:,1,1], kind='cubic',fill_value='extrapolate')
    
    return [M11, M12], [M21,M22]

L1_ipltd_2nd_bi, L2_ipltd_2nd_bi = get_ipltd_function_2nd_bi(some_t, some_ivls, ODE_L_bi_sol_2nd)

# Define interpolated function to avoid doing repeated interpolation
def L_ipltd_2nd_bi(t):
    L1 = np.array([L1_ipltd_2nd_bi[0](t), L1_ipltd_2nd_bi[1](t)])
    L2 = np.array([L2_ipltd_2nd_bi[0](t), L2_ipltd_2nd_bi[1](t)])
    return np.array([L1, L2])


''' Second order mixed-moments Q-lambda - obtained from ODE solver package 
    Note: we need additional wrapper of scipy's odeint for matrix-valued ODEs, namely the _odeintw.py file
    Source: https://github.com/WarrenWeckesser/odeintw'''

def ODE_LQ_bi_sol_2nd(t,ivls):
    t_vec_ODE = np.linspace(0,t,ivls)
    init = np.zeros((2,2))
    
    def ODE_LQ_bi_2nd(QL,u):
        L_01 = L_ipltd_1st_bi(u)
        Q_01 = Q_ipltd_1st_bi(u)
        L_02 = L_ipltd_2nd_bi(u)
        dQLdu = np.dot(B_bi - D_alpha_bi, QL) - np.dot(QL, D_mu_bi) + L_02 + np.outer(L_01_2dim, Q_01) + np.dot(B_bi, np.diag(L_01))
        return dQLdu
                       
    return _odeintw.odeintw(ODE_LQ_bi_2nd, init, t_vec_ODE)


L1Q_ipltd_2nd_bi, L2Q_ipltd_2nd_bi = get_ipltd_function_2nd_bi(some_t, some_ivls, ODE_LQ_bi_sol_2nd)

def LQ_ipltd_2nd_bi(t):
    L1Q = np.array([L1Q_ipltd_2nd_bi[0](t), L1Q_ipltd_2nd_bi[1](t)])
    L2Q = np.array([L2Q_ipltd_2nd_bi[0](t), L2Q_ipltd_2nd_bi[1](t)])
    return np.array([L1Q, L2Q])


''' Second order reduced Q moments - obtained from ODE solver package 
    Note: we need additional wrapper of scipy's odeint for matrix-valued ODEs, namely the _odeintw.py file
    Source: https://github.com/WarrenWeckesser/odeintw'''


def ODE_Q_bi_sol_2nd(t, ivls):
    t_vec_ODE = np.linspace(0,t,M)
    init = np.zeros((2,2))
    
    def ODE_Q_bi_2nd(Q,u):
        LQ = LQ_ipltd_2nd_bi(u)
        dQdu = -np.dot(D_mu_bi, Q) - np.dot(Q, D_mu_bi) + LQ + LQ.T
        return dQdu
    
    return _odeintw.odeintw(ODE_Q_bi_2nd, init, t_vec_ODE)
                  
